<a href="https://colab.research.google.com/github/rosiezou/ssl_3d_recon/blob/master/CadToPointCloudProto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prototype code to randomly sample pointcloud from a CAD file belonging to Shapenet. 
https://towardsdatascience.com/deep-learning-on-point-clouds-implementing-pointnet-in-google-colab-1fd65cd3a263 played an elemental role in understanding how to convert CAD files into pointclouds!

In [ ]:
import numpy as np
import random
import math
!pip install path.py;
from path import Path
import plotly.graph_objects as go

In [ ]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
!unzip -q ModelNet10.zip

path = Path("ModelNet10")

--2021-02-16 04:06:18--  http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Resolving 3dvision.princeton.edu (3dvision.princeton.edu)... 128.112.136.61
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473402300 (451M) [application/zip]
Saving to: ‘ModelNet10.zip’

ModelNet10.zip      100%[===================>] 451.47M  38.5MB/s    in 7.7s    

2021-02-16 04:06:26 (58.3 MB/s) - ‘ModelNet10.zip’ saved [473402300/473402300]



In [ ]:
def read_cad_off(filename):
  # Read CAD files with .off extension.
  with open(filename, 'r') as f:
    if 'OFF' != f.readline().strip():
      print(f'Error: {filename} is a not a file.')
      return false
    num_vertices, num_faces, _ = tuple([int(s) for s in f.readline().strip().split(' ')])
    vertices = [[float(s) for s in f.readline().strip().split(' ')] for ith_vertex in range(num_vertices)]
    faces = [[int(s) for s in f.readline().strip().split(' ')][1:] for ith_face in range(num_faces)]
    return vertices, faces

In [ ]:
# Read the sample CAD file and plot the mesh.
sample_file = path/"bed/train/bed_0001.off"
vertices, faces = read_cad_off(sample_file)

plotting_vertex = np.array(vertices).T
fig = go.Figure(data=[go.Mesh3d(x=plotting_vertex[0], y=plotting_vertex[1], z=plotting_vertex[2], color='lightpink', opacity=0.50)])
fig.show()

In [ ]:
# Same bed plot without the meshes (Points only)
fig = go.Figure(data=[go.Scatter3d(x=plotting_vertex[0], y=plotting_vertex[1], z=plotting_vertex[2],
                                   mode='markers', marker=dict(size=1))])
fig.show()

In [ ]:
# Calculate area for each face
areas = np.zeros((len(faces)))
vertices = np.array(vertices)

def calculate_triangle_area(pt1, pt2, pt3):
  # Calculate the area of the triangle formed by the provided points.
  # Note: This area calculation is for a 3D triangle with Heron's formula
  side_a = np.linalg.norm(pt1 - pt2)
  side_b = np.linalg.norm(pt2 - pt3)
  side_c = np.linalg.norm(pt3 - pt1)
  perimeter = 0.5 * ( side_a + side_b + side_c)
  return max(perimeter * (perimeter - side_a) * (perimeter - side_b) * (perimeter - side_c), 0)**0.5


for i in range(len(areas)):
    areas[i] = calculate_triangle_area(vertices[faces[i][0]], vertices[faces[i][1]], vertices[faces[i][2]])

In [ ]:
# Sample to a fixed number of points in each face.
# The probability of choosing a face is proportional to its area.
# The point distribution should be uniform for each face.
k = 5000
sampled_faces = random.choices(faces, weights=areas, k=k)

# Sample points on the surface of the chosen triangle
def sample_point_on_triangle(pt1, pt2, pt3):
    # barycentric coordinates on a triangle
    # https://mathworld.wolfram.com/BarycentricCoordinates.html
    # Another good reference: https://pharr.org/matt/blog/2019/02/27/triangle-sampling-1.html
    s, t = sorted([random.random(), random.random()])
    f = lambda i: s * pt1[i] + (t-s) * pt2[i] + (1-t) * pt3[i]
    return (f(0), f(1), f(2))

In [ ]:
# Construct pointcloud.
pointcloud = np.zeros((k, 3))
for i in range(len(sampled_faces)):
    pointcloud[i] = (sample_point_on_triangle(vertices[sampled_faces[i][0]], vertices[sampled_faces[i][1]], vertices[sampled_faces[i][2]]))

In [ ]:
# Plot pointcloud.
plotting_pointcloud = np.array(pointcloud).T
fig = go.Figure(data=[go.Scatter3d(x=plotting_pointcloud[0], y=plotting_pointcloud[1], z=plotting_pointcloud[2],
                                   mode='markers', marker=dict(size=1))])
fig.show()

In [ ]:
# Normalize pointcloud by subtracting mean and normalizing all points onta a unit sphere.
norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

# rotation around z-axis
theta = random.random() * 2. * math.pi # rotation angle
rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                       [ math.sin(theta),  math.cos(theta),    0],
                       [0,                             0,      1]])

rot_pointcloud = rot_matrix.dot(pointcloud.T).T

# add some noise
noise = np.random.normal(0, 0.02, (pointcloud.shape))
noisy_pointcloud = rot_pointcloud + noise


In [ ]:
print(np.linalg.norm(norm_pointcloud, axis=1))
print(noisy_pointcloud.shape)

[0.49450287 0.93343782 0.80844627 ... 0.98366799 0.4723227  0.33349325]
(5000, 3)


In [ ]:
plotting_pointcloud = np.array(noisy_pointcloud).T
fig = go.Figure(data=[go.Scatter3d(x=plotting_pointcloud[0], y=plotting_pointcloud[1], z=plotting_pointcloud[2],
                                   mode='markers', marker=dict(size=1))])
fig.show()

Prototyping to validate StyleGAN approach

In [ ]:
!unzip stylegan2-ada-pytorch-main.zip

In [ ]:
cd stylegan2-ada-pytorch-main/

/content/stylegan2-ada-pytorch-main


In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In [ ]:
# Generate curated MetFaces images without truncation (Fig.10 left)
!python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

# Generate uncurated MetFaces images with truncation (Fig.12 upper left)
!python generate.py --outdir=out --trunc=0.7 --seeds=600-605 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

# Generate class conditional CIFAR-10 images (Fig.17 left, Car)
!python generate.py --outdir=out --seeds=0-35 --class=1 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/cifar10.pkl

# Style mixing example
!python style_mixing.py --outdir=out --rows=85,100,75,458,1500 --cols=55,821,1789,293 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

In [ ]:
!python legacy.py \
    --source=https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-cat-config-f.pkl \
    --dest=stylegan2-cat-config-f.pkl

In [ ]:
## extracting StyleGAN2 pre-trained model parameters
## must upload ffhq.pkl to runtime environment first
import torch
import pickle
with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module
z = torch.randn([1, G.z_dim]).cuda()    # latent codes
c = None                                # class labels (not used in this example)
img = G(z, c)                           # NCHW, float32, dynamic range [-1, +1]

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [ ]:
import tensorflow as tf

In [ ]:
!unzip frustum-pointnets-master.zip

In [ ]:
cd frustum-pointnets-master/

/content/frustum-pointnets-master


In [ ]:
!pip install pickle-mixin

In [ ]:
!sh scripts/command_prep_data.sh